# Fill test data glosses/examples with LLMs

In [1]:
import pandas as pd
import json
import os
import openai
from dotenv import load_dotenv
import time
import re
import unicodedata
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

## 1. Load the data and compute the number of requests to make

In [2]:
FILE_TO_READ = './data/augmented/axolotl.test.surprise.tsv'
RESPONSES_FILE_1 = './data/augmented/llm-responses/responses-de-1.json'
RESPONSES_FILE_2 = './data/augmented/llm-responses/responses-de-2.json'
AUGMENTED_EXAMPLES = 3

language = FILE_TO_READ.split('.')[-2]
filename = FILE_TO_READ.split('/')[-1].split('.')[0:-1]
filename = '.'.join(filename)
language, filename

('surprise', 'axolotl.test.surprise')

The following requets will be done:
1. For each glos-known row that doesn't have an example available, an example will be generated (+3)**
2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)
3. For each gloss-known row, at least three examples are needed, so if a sense has only one example, two more are generated (+2)

** To avoid repeating requests, the generation of step 3. is also done in step 1. directly for non-example rows.

In [3]:
df = pd.read_csv(FILE_TO_READ, sep='\t')
df.info()
print("\nColumns to fill 'example':" , df['example'].isna().sum(), "(output ×3)")
print("Columns to fill 'gloss':" , df['gloss'].isna().sum(), "(output ×1)")
print("Columns to augment:" , df[~df['gloss'].isna()].shape[0]-df['example'].isna().sum(), "(output ×2)")
print("Total:", df['example'].isna().sum() + df['gloss'].isna().sum() + df[~df['gloss'].isna()].shape[0], "requests")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   usage_id              1152 non-null   object
 1   word                  1152 non-null   object
 2   orth                  1152 non-null   object
 3   sense_id              584 non-null    object
 4   gloss                 584 non-null    object
 5   example               1152 non-null   object
 6   indices_target_token  1152 non-null   object
 7   date                  1152 non-null   int64 
 8   period                1152 non-null   object
dtypes: int64(1), object(8)
memory usage: 81.1+ KB

Columns to fill 'example': 0 (output ×3)
Columns to fill 'gloss': 568 (output ×1)
Columns to augment: 584 (output ×2)
Total: 1152 requests


## 2. Define methods to send and store requests to OpenAI's API

In [4]:
load_dotenv('./gpt35.env')
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_ENDPOINT")
engine=os.getenv("OPENAI_IMPLEMENTATION")
model=os.getenv("MODEL_NAME")
model

'gpt-3.5-turbo'

In [5]:
MAX_ITERATIONS = 1 # only one repetition is allowed
def request(prompt, max_tokens=500, temperature=0, iterations=0):
    """Request a completion from the OpenAI API.
    :param prompt: The prompt to send to the API
    :param max_tokens: The maximum number of tokens in the output
    :param temperature: The degree of randomness in the output
    :return response: The response from the API
    :return usage: The count of token usage from the API { "input", "output" }
    """
    try:
        response = openai.ChatCompletion.create(
            api_key=api_key,
            api_base=api_base,
            engine=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}],
        )
        finish_reason = response["choices"][0]["finish_reason"]
        rsp = "" if finish_reason == "content_filter" else response["choices"][0]["message"]["content"]
        return rsp, { "input": response["usage"]["prompt_tokens"], "output": response["usage"]["completion_tokens"] }, finish_reason
    except Exception as e:
        response = ""
        usage = {"input": 0, "output": 0}
        if f"{e}".startswith("The response was filtered due to the prompt triggering Azure OpenAI's content management policy"):
            finish_reason = "content_filter"
        else:
            if ("Max retries exceeded with url" in f"{e}") and (iterations < MAX_ITERATIONS):
                    print("MAX_RETRIES triggered. Retrying in 60 seconds...")
                    time.sleep(60)
                    return request(prompt, max_tokens, temperature, iterations+1) # retry after 60 seconds
            finish_reason = f"ERROR [{type(e).__name__}]: {e}"
        return response, usage, finish_reason

gen_prompt = lambda prompt, *args: prompt.format(*args)

Finally, a method to find the target word within the sentence (wether the "word" or the "orth"):

In [6]:
def generate_substrings(word):
    substrings = []
    for i in range(len(word), 0, -1):
        substrings.append(word[:i])
    return substrings[1:-1]

def find_sub_list(sl,l): # not used because some examples have no exact coincidence
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind+1,ind+sll # +1 for the [CLS] token

def extract_letters(input_string):
    return re.sub(r'[^а-яА-Яa-zA-ZÀ-ÿёЁ\u0300-\u036f-]', '', input_string)

def remove_accents(word):
    normalized_word = unicodedata.normalize('NFD', word)
    cleaned_word = re.sub(r'[\u0300-\u036f]', '', normalized_word)
    return cleaned_word

def find_word_containing_target(sentence, target_word):
    index = sentence.find(target_word)
    if index == -1:
        return None
    start_index = sentence.rfind(" ", 0, index) + 1 if index != 0 else 0
    end_index = sentence.find(" ", index + len(target_word)) if sentence.find(" ", index + len(target_word)) != -1 else len(sentence)
    final_char = final_char = " " if target_word.endswith(" ") else ""
    return sentence[start_index:end_index].split()[0] + final_char

def get_search(example, word, orth=None, print_search=False):
    # append the words to search in the example, in the desired ORDER
    # 1 - the word (with an ending character), and the word itself
    search = [f"{word} ", f"{word},", f"{word}.", word]

    # 2 - the orthographic form (with an ending character), and the orthographic form itself
    if orth and orth != word:
        search += [f"{orth} ", f"{orth},", f"{orth}.", orth]

    # 3 - all substrings of the word (i.e. выходить -> ['выходит', 'выходи', 'выход', 'выхо', 'вых', 'вы'])
    search += generate_substrings(word)

    # 4 - all substrings of the orthographic form
    if orth:
        search.extend([i for i in generate_substrings(orth) if i not in search])

    # 5 - the word without accents (with an ending character), and the word without accents itself
    unicoded_word = remove_accents(word)
    if unicoded_word != word:
        search += [f"{unicoded_word} ", f"{unicoded_word},", f"{unicoded_word}.", unicoded_word]
    
    # 6 - the orthographic form without accents (with an ending character), and the orthographic form without accents itself
    if orth and orth != word:
        unicoded_orth = remove_accents(orth)
        if unicoded_orth != orth:
            search += [f"{unicoded_orth} ", f"{unicoded_orth},", f"{unicoded_orth}.", unicoded_orth]

    # 7 - all substrings of the word without accents
    if unicoded_word != word:
        search.extend([i for i in generate_substrings(unicoded_word) if i not in search])

    # 8 - all substrings of the orthographic form without accents
    if orth and orth != word and unicoded_orth != orth:
        search.extend([i for i in generate_substrings(unicoded_orth) if i not in search])

    if print_search:
        print(f"Searching for: {search}")

    # FIND the first search-string that is within the example, if any (in upper or lowercase)
    for s in search:
        search_word = find_word_containing_target(example, s)
        if search_word:
            break
        search_word = find_word_containing_target(example.lower(), s.lower())
        if search_word:
            index = example.lower().find(search_word)
            if index == -1:
                # this should never happen
                raise Exception(f"Found '{search_word}' in '{example.lower()}', but then not found...")
            else:
                search_word = example[index:index + len(search_word)]
            break
    else:
        search_word = ""
    return extract_letters(search_word)

In [7]:
print(f'Find {(df.loc[0, "word"], df.loc[0, "orth"])} in the sentence: {df.loc[0, "example"]}')
result_example = get_search(df.loc[0, "example"], df.loc[0, "word"], df.loc[0, "orth"], print_search=True)
print(f"Found: {result_example}" if result_example else "Not found")

Find ('zersetzen', 'zersetzen') in the sentence: Er vollendete seine Attacke mit einem Satz, in dem Nazijargon als.volkstümlicher Sprachgebrauch bezeichnet wirdr.. Es ist nicht uninteressant, daß im Zusammenhang mit den Intellektuellen lm volkstümlichen-Sprachgebrauch die Wörtchen. zersetzend ' und. entwurzelnd ' immer wieder auftauchen.
Searching for: ['zersetzen ', 'zersetzen,', 'zersetzen.', 'zersetzen', 'zersetze', 'zersetz', 'zerset', 'zerse', 'zers', 'zer', 'ze']
Found: zersetzend


## 3. Prepare and send requests for each of the 3 cases (2 in the same)

### 1. For each glos-known row that doesn't have an example available, an example will be generated (+1)
&&
### 3. For each gloss-known row, two more examples will be generated (data augmentation) (+2)
Additionally, in this step, the data is augmented: an extra-2 examples are generated for each request (3 in total)

#### 1.1. Prepare dataset and prompt

In [8]:
df[~df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]

,word,orth,sense_id,gloss,example
1,zersetzen,zersetzen,zersetzen_sense0,"eine schädigende, zerstörende Wirkung auf den ...",Die Hauptpunkte sind a) Böhmisch und Sächsisch...
2,zersetzen,zersetzen,zersetzen_sense1,"etw., sich auflösen","Das Wasser, das durch den Kalkstein sickert un..."
3,zersetzen,zersetzen,zersetzen_sense1,"etw., sich auflösen","Solcher Humus, wenigstens der tiefer liegende,..."
5,zersetzen,zersetzen,zersetzen_sense1,"etw., sich auflösen","Nach den bisherigen Gründen vermuthe ich, dass..."
6,zersetzen,zersetzen,zersetzen_sense0,"eine schädigende, zerstörende Wirkung auf den ...",Je verwickelter nun auch im Mutterlande die Le...
...,...,...,...,...,...
1141,Manschette,Manschette,Manschette_sense1,(steifer) zuknöpfbarer Abschluss eines langen ...,Herr Waser aber klopfte den Staub des Handgeme...
1145,Manschette,Manschette,Manschette_sense1,(steifer) zuknöpfbarer Abschluss eines langen ...,"Racine mußte große Kränkungen erleben, große K..."
1148,Manschette,Manschette,Manschette_sense1,(steifer) zuknöpfbarer Abschluss eines langen ...,"Jede der beiden Manschetten hat am Saume, wo s..."
1150,Manschette,Manschette,Manschette_sense1,(steifer) zuknöpfbarer Abschluss eines langen ...,"Sie sind sehr gebildet, denn die Dame singt al..."


In [9]:
# original (english): 'Give me ONLY {} line-separated usage examples of the word "{}" with the following definition:\n{}'

if language == "ru":
    prompt = 'Приведите мне ТОЛЬКО {} примера употребления слова "{}" в строках со следующим определением:\n{}'
elif language == "fi":
    prompt = 'Anna minulle VAIN {} rivieroteltua esimerkkiä sanasta "{}" seuraavalla määritelmällä:\n{}'
else: # language == "de"
    prompt = 'Geben Sie mir NUR {} durch Zeilen getrennte Verwendungsbeispiele für das Wort „{}“ mit der folgenden Definition:\n{}'

exp_fill = df[~df['gloss'].isna()]
for i in range(15):
    print(prompt.format(AUGMENTED_EXAMPLES, exp_fill.iloc[i, 1], exp_fill.iloc[i, 4]), end="\n\n")
del exp_fill

Geben Sie mir NUR 3 durch Zeilen getrennte Verwendungsbeispiele für das Wort „zersetzen“ mit der folgenden Definition:
eine schädigende, zerstörende Wirkung auf den Bestand von etw. ausüben, etw. untergraben

Geben Sie mir NUR 3 durch Zeilen getrennte Verwendungsbeispiele für das Wort „zersetzen“ mit der folgenden Definition:
etw., sich auflösen

Geben Sie mir NUR 3 durch Zeilen getrennte Verwendungsbeispiele für das Wort „zersetzen“ mit der folgenden Definition:
etw., sich auflösen

Geben Sie mir NUR 3 durch Zeilen getrennte Verwendungsbeispiele für das Wort „zersetzen“ mit der folgenden Definition:
etw., sich auflösen

Geben Sie mir NUR 3 durch Zeilen getrennte Verwendungsbeispiele für das Wort „zersetzen“ mit der folgenden Definition:
eine schädigende, zerstörende Wirkung auf den Bestand von etw. ausüben, etw. untergraben

Geben Sie mir NUR 3 durch Zeilen getrennte Verwendungsbeispiele für das Wort „zersetzen“ mit der folgenden Definition:
etw., sich auflösen

Geben Sie mir NUR 3 du

In [10]:
r = {"data":[], "checkpoint": 0, "input_tokens": 0, "output_tokens": 0}
if os.path.exists(RESPONSES_FILE_1):
    with open(RESPONSES_FILE_1, "r") as f:
        r = json.load(f)
else:
    with open(RESPONSES_FILE_1, "w") as f:
        f.write(json.dumps(r, indent=4))

#### 1.2. Send requests

In [11]:
def generate_examples(n, prompt):
    response, usage, finish_reason = request(prompt)
    print(response)

    status = "ok"
    lines = response.split('\n')
    if finish_reason != "stop":
        status = "error"
        new_lines = [f"Error {finish_reason}" for _ in range(n)]
    else:
        if n == 1:
            if len(lines) > 1:
                if len(lines) > 2:
                    print(f"WARNING 1: {len(lines)} lines found, but {n} expected.")
                    status = "warning"
                new_lines = [lines[-1]]
            new_lines = lines
        else:
            new_lines = []
            for line in lines:
                if (line != "") and ((line[0].isdigit()) or line[0] == '-'):
                    line = ' '.join(line.split(' ')[1:])
                    new_lines.append(line)
            if len(new_lines) != n:
                print(f"WARNING 2: {len(new_lines)} lines found, but {n} expected.")
                status = "warning"

    r['input_tokens'] += usage['input']
    r['output_tokens'] += usage['output']
    r['data'].append({"prompt":prompt, "response":response, "finish_reason":finish_reason, "status":status})
    with open(RESPONSES_FILE_1, "w", encoding='utf-8') as f:
        f.write(json.dumps(r, indent=4))

    return new_lines

In [12]:
df = pd.read_csv(FILE_TO_READ, sep='\t')
mod_df = df.copy()
id_template = '_'.join(df.loc[0, 'usage_id'].split("_")[:2]) + '_'
last_id = int(df.loc[len(df)-1, 'usage_id'].split("_")[-1])

index = 0

In [14]:
while index < len(df):
    row = mod_df.iloc[index]
    gloss = row['gloss']
    if f"{gloss}" != "nan":
        sense_id = row['sense_id']
        word = row['word']
        example = row['example']
        print(f"---------------------------------------- {index, sense_id} ----------------------------------------")

        found = len(mod_df[(mod_df['sense_id'] == sense_id) & (~mod_df['example'].isna())])
        n = AUGMENTED_EXAMPLES - found
        if n > 0:
            p = gen_prompt(prompt, n, word, gloss)
            print(p, end="\n\n")
            examples = generate_examples(n, p)
            if f"{example}" == "nan":
                row['example'] = examples[0]
                mod_df.iloc[index] = row
                start_at = 1
            else:
                start_at = 0
            for e in examples[start_at:]:
                last_id += 1
                row['example'] = e
                row['usage_id'] = f"{id_template}_{last_id}"
                mod_df.loc[len(mod_df)] = row
        else:
            print(f"No examples needed for {word} ({sense_id}), found {found}")
    index += 1

mod_df.to_csv(f'./data/augmented/{filename}.tsv', sep='\t', index=False)
mod_df

---------------------------------------- (972, 'abgebrüht_sense2') ----------------------------------------
Geben Sie mir NUR 2 durch Zeilen getrennte Verwendungsbeispiele für das Wort „abgebrüht“ mit der folgenden Definition:
gefühllos, frech, dickfellig, abgehärtet gegen sittl. Eindrücke etc.

1. Der abgebrühte Verkäufer ließ sich von den Beschwerden der Kunden nicht beeindrucken und blieb frech und ungerührt.
2. Trotz der schockierenden Bilder blieb der abgebrühte Journalist gefühllos und berichtete nüchtern über die schrecklichen Ereignisse.
---------------------------------------- (973, 'abgebrüht_sense1') ----------------------------------------
No examples needed for abgebrüht (abgebrüht_sense1), found 24
---------------------------------------- (974, 'abgebrüht_sense1') ----------------------------------------
No examples needed for abgebrüht (abgebrüht_sense1), found 24
---------------------------------------- (975, 'abgebrüht_sense1') ----------------------------------------


,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,test_surprise_0,zersetzen,zersetzen,NaN,NaN,"Er vollendete seine Attacke mit einem Satz, in...",234:244,1965,new
1,test_surprise_1,zersetzen,zersetzen,zersetzen_sense0,"eine schädigende, zerstörende Wirkung auf den ...",Die Hauptpunkte sind a) Böhmisch und Sächsisch...,211:221,1855,old
2,test_surprise_2,zersetzen,zersetzen,zersetzen_sense1,"etw., sich auflösen","Das Wasser, das durch den Kalkstein sickert un...",456:464,1859,old
3,test_surprise_3,zersetzen,zersetzen,zersetzen_sense1,"etw., sich auflösen","Solcher Humus, wenigstens der tiefer liegende,...",153:161,1810,old
4,test_surprise_4,zersetzen,zersetzen,NaN,NaN,"Sie unternimmt den untauglichen Versuch, zwisc...",252:261,1973,new
...,...,...,...,...,...,...,...,...,...
1185,test_surprise__1185,Titel,Titel,Titel_sense3,jur. Abschnitt eines Gesetzes,- Der Titel des Bürgerlichen Gesetzbuches rege...,121:127,1840,old
1186,test_surprise__1186,Titel,Titel,Titel_sense3,jur. Abschnitt eines Gesetzes,- Im Titel 3 des Strafgesetzbuches werden die ...,121:127,1840,old
1187,test_surprise__1187,Titel,Titel,Titel_sense3,jur. Abschnitt eines Gesetzes,- Der Titel des Handelsgesetzbuches enthält di...,121:127,1840,old
1188,test_surprise__1188,Titel,Titel,Titel_sense3,jur. Abschnitt eines Gesetzes,- Im Titel 5 des Arbeitsgesetzbuches sind die ...,121:127,1840,old


In [16]:
for i in r['data']:
    if i['status'] == 'warning':
        print("-------------------------------------------")
        print(i['prompt'])
        print()
        print(i["response"])

-------------------------------------------
Geben Sie mir NUR 1 durch Zeilen getrennte Verwendungsbeispiele für das Wort „Seminar“ mit der folgenden Definition:
Form der Lehrveranstaltung, besonders an Hochschulen, bei der das Studium in einer Arbeitsgemeinschaft unter der Leitung eines Wissenschaftlers oder Fachmannes durchgeführt wird

- Das Seminar "Einführung in die Literaturwissenschaft" wird von Professor Müller geleitet.
- Im Seminar "Grundlagen der Psychologie" werden verschiedene Experimente durchgeführt.
- Das Seminar "Marketingstrategien" bietet den Studierenden die Möglichkeit, ihre eigenen Ideen für Werbekampagnen zu entwickeln.
- Im Seminar "Internationales Recht" werden aktuelle Fälle aus der Praxis diskutiert.
- Das Seminar "Kreatives Schreiben" ermöglicht den Teilnehmern, ihre eigenen Texte zu verfassen und zu besprechen.
-------------------------------------------
Geben Sie mir NUR 1 durch Zeilen getrennte Verwendungsbeispiele für das Wort „Rezeption“ mit der folgende

### 2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)

#### 2.1. Prepare dataset and prompt

In [17]:
df[df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]

,word,orth,sense_id,gloss,example
0,zersetzen,zersetzen,NaN,NaN,"Er vollendete seine Attacke mit einem Satz, in..."
4,zersetzen,zersetzen,NaN,NaN,"Sie unternimmt den untauglichen Versuch, zwisc..."
9,zersetzen,zersetzen,NaN,NaN,"Bakterien, die in relativ kurzer Zeit bei Hava..."
10,zersetzen,zersetzen,NaN,NaN,Doch der Antikommunismus Ist auch heute wieder...
11,zersetzen,zersetzen,NaN,NaN,Diese „westliche Scheinkultur zersetzt und zer...
...,...,...,...,...,...
1143,Manschette,Manschette,NaN,NaN,Selbst wenn noch kein Fettaustritt sichtbar is...
1144,Manschette,Manschette,NaN,NaN,Am Mode-Il link « oben sind es der Einsatz und...
1146,Manschette,Manschette,NaN,NaN,68000 Manschetten zusätzlich — Ergebnis seiner...
1147,Manschette,Manschette,NaN,NaN,"Die Manschetten sind etwa denen ähnlich, die b..."


In [18]:
# original (english): 'What does "{}" mean in the following examples (ONLY return JSON with the key "definition" - detailed definition)?\n{}'

if language == "ru":
    prompt = 'Что означает «{}» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?\n{}'
elif language == "fi":
    prompt = 'Mitä «{}» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?\n{}'
else: # language == "de"
    prompt = 'Was bedeutet „{}“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?\n{}'

gls_fill = df[df['gloss'].isna()]
for i in range(5):
    print(prompt.format(gls_fill.iloc[i, 1], gls_fill.iloc[i, 5]), end="\n\n")
del gls_fill

Was bedeutet „zersetzen“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?
Er vollendete seine Attacke mit einem Satz, in dem Nazijargon als.volkstümlicher Sprachgebrauch bezeichnet wirdr.. Es ist nicht uninteressant, daß im Zusammenhang mit den Intellektuellen lm volkstümlichen-Sprachgebrauch die Wörtchen. zersetzend ' und. entwurzelnd ' immer wieder auftauchen.

Was bedeutet „zersetzen“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?
Sie unternimmt den untauglichen Versuch, zwischen den sozialistischen Ländern zu differenzieren, Mißtrauen zwischen ihnen zu säen und das feste Bündnis der sozialistischen Bruderländer sozusagen durch eine Sonderbehandlung Jedes einzelnen von ihnen zu zersetzen.

Was bedeutet „zersetzen“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?
Bakterien, die in relativ k

In [19]:
r2 = {"data":[], "checkpoint": 0, "input_tokens": 0, "output_tokens": 0}
if os.path.exists(RESPONSES_FILE_2):
    with open(RESPONSES_FILE_2, "r") as f:
        r2 = json.load(f)
else:
    with open(RESPONSES_FILE_2, "w") as f:
        f.write(json.dumps(r2, indent=4))

#### 2.2. Send requests

In [21]:
def generate_gloss(prompt):
    response, usage, finish_reason = request(prompt)
    print(response)

    status = "ok"

    if finish_reason != "stop":
        status = "error"
        gloss = f"Error {finish_reason}"
    else:
        json_match = re.search(r'{.*}', response, re.DOTALL)
        if json_match:
            json_string = json_match.group(0)
            try:
                glosses = list(json.loads(json_string).values())
            except json.JSONDecodeError as e:
                if language == "ru":
                    pattern = r'(?<="определение": ").*(?=")'
                else:
                    pattern = r'(?<="definition": ").*(?=")'
                definition = re.search(pattern, json_string).group()
                if definition:
                    glosses = ['"' + definition]
                else:
                    raise e
        else:
            print("No JSON found in the input string.")
            status = "ok_nojson"
            lines = response.split('\n')
            if len(lines) > 1:
                print(f"WARNING 1: {len(lines)} lines found, but 1 expected.")
                status = "warning"
                glosses = [lines[-1]]
            else:
                glosses = [response]
        if len(glosses) > 1:
            print(f"WARNING 2: {len(glosses)} glosses found, but 1 expected.")
            status = "warning"
        gloss = glosses[-1]
    
    r2['input_tokens'] += usage['input']
    r2['output_tokens'] += usage['output']
    r2['data'].append({"prompt":prompt, "response":response, "finish_reason":finish_reason, "status":status})
    with open(RESPONSES_FILE_2, "w", encoding='utf-8') as f:
        f.write(json.dumps(r2, indent=4))

    if language == "ru":
        gloss = gloss.replace("в данном контексте означает", "-")
    # fi : tarkoittaa
    # de : In diesem Kontext bedeutet
    return gloss

In [22]:
mod_df = pd.read_csv(f'./data/augmented/{filename}.tsv', sep='\t')
df_2 = mod_df.copy()
mod_df_2 = df_2.copy()

for index, row in mod_df_2.iterrows():
    gloss = row["gloss"]
    if f"{gloss}" == "nan":
        example = row["example"]
        word = row["word"]
        orth = row["orth"]
        s = get_search(example, word, orth)
        print(f"---------------------------------------- {index, s} ----------------------------------------")
        if s == "":
            s = word
        p = gen_prompt(prompt, s, example)
        print(p, end="\n\n")
        try:
            gloss = generate_gloss(p)
        except json.JSONDecodeError:
            print("DECODING ERROR - TERMINATING")
            break
        row['gloss'] = gloss
        mod_df_2.iloc[index] = row

mod_df_2.to_csv(f'./data/augmented/{filename}.tsv', sep='\t', index=False)
mod_df_2

---------------------------------------- (0, 'zersetzend') ----------------------------------------
Was bedeutet „zersetzend“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?
Er vollendete seine Attacke mit einem Satz, in dem Nazijargon als.volkstümlicher Sprachgebrauch bezeichnet wirdr.. Es ist nicht uninteressant, daß im Zusammenhang mit den Intellektuellen lm volkstümlichen-Sprachgebrauch die Wörtchen. zersetzend ' und. entwurzelnd ' immer wieder auftauchen.

In den gegebenen Beispielen bedeutet "zersetzend" etwas, das den Geist oder die Moral einer Person negativ beeinflusst oder schwächt. Es bezieht sich auf etwas, das destabilisierend oder destruktiv wirkt und dazu führen kann, dass jemand seine Überzeugungen, Werte oder Identität verliert.
No JSON found in the input string.
---------------------------------------- (4, 'zersetzen') ----------------------------------------
Was bedeutet „zersetzen“ in den folgenden Be

In [ ]:
for i in r2['data']:
    if i['status'] == 'warning':
        print("-------------------------------------------")
        print(i['prompt'])
        print()
        print(i["response"])

-------------------------------------------
Mitä «Cala-weden» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?
Jos jocu, Calan-pyyndö paicasa tahdolas turmele toisen Huoneen, Cala-weden, Aluxen

Cala-weden tarkoittaa seuraavissa esimerkeissä:

1. "Cala-weden" tarkoittaa Aluxenin tapauksessa toisen Huoneen turmelemista Calan-pyynnön seurauksena.
-------------------------------------------
Mitä «hän» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?
Cuca coiran hännän nosta, jos ei hän itze

JSON avaimella "definition" palautettu monisanainen määritelmä:

"Hän" tarkoittaa tässä yhteydessä henkilöä, joka ei itse nosta Cuca coiran häntää.
-------------------------------------------
Mitä «Juuresa» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?
Juuresa heinän sato on

"Juuresa" tarkoittaa seuraavissa esimerkeissä:

1. "Juu